In [1]:
import numpy as np

In [2]:
import pandas as pd

In [3]:
train_data = pd.read_csv("C:/Users/amans/OneDrive/Documents/All Machine Learning/ML Dataset/train.csv", encoding= 'unicode_escape')

In [4]:
test_data = pd.read_csv("C:/Users/amans/OneDrive/Documents/All Machine Learning/ML Dataset/test.csv",encoding= 'unicode_escape')

In [5]:
import tensorflow as tf
print(tf.__version__)

2.3.0


In [6]:
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
%matplotlib inline

In [7]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_data['sentiment'] = le.fit_transform(train_data['sentiment'])
test_data['sentiment'] = le.fit_transform(test_data['sentiment'])

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB # classifier 
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    plot_confusion_matrix,
)
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
# text preprocessing modules
from string import punctuation 
# text preprocessing modules
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
import re

In [9]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\amans\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [10]:
stop_words =  stopwords.words('english')
def text_cleaning(text, remove_stop_words=True, lemmatize_words=True):
    # Clean the text, with the option to remove stop_words and to lemmatize word
    # Clean the text
    text = re.sub(r"[^A-Za-z0-9]", " ", text)
    text = re.sub(r"\'s", " ", text)
    text =  re.sub(r'http\S+',' link ', text)
    text = re.sub(r'\b\d+(?:\.\d+)?\s+', '', text) # remove numbers
        
    # Remove punctuation from text
    text = ''.join([c for c in text if c not in punctuation])
    
    # Optionally, remove stop words
    if remove_stop_words:
        text = text.split()
        text = [w for w in text if not w in stop_words]
        text = " ".join(text)
    
    # Optionally, shorten words to their stems
    if lemmatize_words:
        text = text.split()
        lemmatizer = WordNetLemmatizer() 
        lemmatized_words = [lemmatizer.lemmatize(word) for word in text]
        text = " ".join(lemmatized_words)
    
    # Return a list of words
    return(text)

In [11]:
import sys
sys.version

'3.8.8 (default, Feb 24 2021, 15:54:32) [MSC v.1928 64 bit (AMD64)]'

In [12]:
train_data.head()

,textID,text,selected_text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²)
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",1,morning,0-20,Afghanistan,38928346,652860.0,60
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,0,noon,21-30,Albania,2877797,27400.0,105
2,088c60f138,my boss is bullying me...,bullying me,0,night,31-45,Algeria,43851044,2381740.0,18
3,9642c003ef,what interview! leave me alone,leave me alone,0,morning,46-60,Andorra,77265,470.0,164
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",0,noon,60-70,Angola,32866272,1246700.0,26


In [13]:
test_data.shape

(4815, 9)

In [14]:
train_data = train_data.dropna()
test_data = test_data.dropna()

In [15]:
train_input = []
train_label = []
test_label = []
test_input = []
for j in range(0,3534):
    c = test_data.iloc[j:j+1,1:2]
    d = test_data.iloc[j:j+1,2:3]
    test_input.append(str(c))
    test_label.append(d)

In [16]:
testing_labels_final = np.array(test_label)

In [17]:
train_input1 = []
train_label1 = []
for i in range(0,27480):
    a = train_data.iloc[i:i+1,1:2]
    b = train_data.iloc[i:i+1,3:4]
    train_input1.append(str(a))
    train_label1.append(b)

In [18]:
training_labels_final1 = np.array(train_label1)

In [19]:
vocab_size = 4000
embedding_dim = 80
max_length = 120
trunc_type = 'post'
oov_tok = "<OOV>"

In [20]:
tokenizer = Tokenizer(num_words = vocab_size,oov_token=oov_tok)
tokenizer.fit_on_texts(train_input1)
word_index = tokenizer.word_index
sequences1 = tokenizer.texts_to_sequences(train_input1)
padded1 = pad_sequences(sequences1,maxlen=max_length,truncating=trunc_type)

In [21]:
testing_sequences1 = tokenizer.texts_to_sequences(test_input)
testing_padded1 = pad_sequences(testing_sequences1,maxlen=max_length)

In [22]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(192)),
    tf.keras.layers.Dense(72, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
]
)
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 80)           320000    
_________________________________________________________________
bidirectional (Bidirectional (None, 384)               419328    
_________________________________________________________________
dense (Dense)                (None, 72)                27720     
_________________________________________________________________
dense_1 (Dense)              (None, 16)                1168      
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 51        
Total params: 768,267
Trainable params: 768,267
Non-trainable params: 0
_________________________________________________________________


In [ ]:
num_epochs = 20
history = model.fit(padded1, training_labels_final1, epochs=num_epochs, validation_data=(testing_padded1, testing_labels_final))

Epoch 1/20
859/859 [==============================] - 150s 175ms/step - loss: 0.8853 - accuracy: 0.5813 - val_loss: 0.8064 - val_accuracy: 0.6327
Epoch 2/20
859/859 [==============================] - 149s 173ms/step - loss: 0.7396 - accuracy: 0.6781 - val_loss: 0.7827 - val_accuracy: 0.6474
Epoch 3/20
859/859 [==============================] - 148s 173ms/step - loss: 0.6777 - accuracy: 0.7057 - val_loss: 0.8092 - val_accuracy: 0.6370
Epoch 4/20
859/859 [==============================] - 145s 168ms/step - loss: 0.6230 - accuracy: 0.7295 - val_loss: 0.8498 - val_accuracy: 0.6338
Epoch 5/20
859/859 [==============================] - 145s 169ms/step - loss: 0.5702 - accuracy: 0.7525 - val_loss: 0.9306 - val_accuracy: 0.6304
Epoch 6/20
820/859 [===========================>..] - ETA: 6s - loss: 0.5211 - accuracy: 0.7721

In [ ]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()

plot_graphs(history, 'accuracy')
plot_graphs(history, 'loss')

In [ ]:
model.save('Sentiment_Analysis')